In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

revs = pickle.load(open('pre2','rb'))
max_fatures = 70000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(revs)
X1 = tokenizer.texts_to_sequences(revs)
X1 = pad_sequences(X1)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yogesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
model = load_model('model2.h5')
model.summary()

C:\Users\Yogesh\Anaconda3\envs\yrenv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1360, 150)         10500000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1360, 150)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               280800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 10,781,202
Trainable params: 10,781,202
Non-trainable params: 0
_________________________________________________________________


In [5]:
def analyze(str,model=model):
    rev = re.sub('[^a-zA-Z]', ' ',str)
    rev = rev.lower()
    rev = rev.split()
    ps = PorterStemmer()
    rev = [ps.stem(word) for word in rev]
    qq = tokenizer.texts_to_sequences(rev)
    lst = [item for elem in qq for item in elem]
    pp = []
    pp.append(lst)
    padded = pad_sequences(pp,maxlen=1360)
    pred = model.predict(padded)
    print("Negative :",pred[0][0],"\tPositive :",pred[0][1])

In [6]:
analyze("This teacher is worst",model)

Negative : 0.9580685 	Positive : 0.041931476


In [74]:
analyze("This teacher has very good teaching skills",model)

Negative : 0.012083446 	Positive : 0.9879166


In [77]:
analyze("This teacher has great presentation skills")

Negative : 0.011755181 	Positive : 0.9882448


In [78]:
analyze("This teacher did not complete syllabus in time")

Negative : 0.7152998 	Positive : 0.28470024


In [79]:
analyze("This teacher completed syllabus in time")

Negative : 0.084921494 	Positive : 0.9150785


In [80]:
analyze("this teacher is very strict",model)

Negative : 0.14615095 	Positive : 0.853849


In [81]:
analyze("The proffessor is well versed in his subject and is very experienced when it comes to the practical part but he is very bad at sharing that knowledge with us.")

Negative : 0.2994336 	Positive : 0.7005665


In [82]:
analyze("This course is easy to grasp and learn , the main problem is that it gets very boring after a certain point and becomes uninteresting")

Negative : 0.84262145 	Positive : 0.1573786


In [55]:
analyze(" is that it gets very boring and  after a certain point")

[[10, 18, 8, 75, 12, 448, 2, 218, 6, 761, 199]]


array([[0.7403249 , 0.25967515]], dtype=float32)

In [83]:
analyze("The professor has great teaching skills")

Negative : 0.0095181605 	Positive : 0.9904819


In [90]:
analyze("the teacher does not teach good but helps in personal matters")

Negative : 0.25388896 	Positive : 0.74611104


In [91]:
analyze("The proffesor lacks practical knowledge")

Negative : 0.8516091 	Positive : 0.14839087


In [92]:
analyze("this is the best professor i have ever has in my entire life")

Negative : 0.0030896745 	Positive : 0.9969104
